In [1]:
import json
import logging
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any, Optional

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DoubleType, LongType, ShortType, ByteType, BinaryType, BooleanType, DateType, TimestampType, NullType, ArrayType, MapType
from pyspark.sql.functions import col, lit, current_timestamp

class YelpDataProcessor:
    def __init__(self, spark_config: Dict[str, str] = None):
        """
        Khởi tạo YelpDataProcessor với cấu hình Spark tùy chỉnh
        
        Args:
            spark_config: Dictionary chứa các cấu hình Spark
        """
        self.logger = self._setup_logging()
        self.spark = self._create_spark_session(spark_config)
        
    def _setup_logging(self) -> logging.Logger:
        """Cấu hình logging chi tiết hơn"""
        logging.basicConfig(
            level=logging.DEBUG,  # Thay đổi level thành DEBUG để có thêm thông tin
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        )
        return logging.getLogger(__name__)

    def _create_spark_session(self, spark_config: Dict[str, str] = None) -> SparkSession:
        """
        Tạo SparkSession với cấu hình tùy chỉnh và logging
        """
        default_config = {
            "spark.sql.parquet.compression.codec": "snappy",
            "spark.sql.parquet.mergeSchema": "true",
            "spark.sql.files.maxPartitionBytes": "128MB",
            "spark.sql.shuffle.partitions": "10",  # Giảm số partition mặc định
            "spark.driver.memory": "2g",
            "spark.executor.memory": "2g"
        }
        
        builder = SparkSession.builder.appName("YelpDataProcessor")
        
        # Áp dụng cấu hình mặc định
        for key, value in default_config.items():
            builder = builder.config(key, value)
            
        # Ghi đè bằng cấu hình tùy chỉnh nếu có
        if spark_config:
            for key, value in spark_config.items():
                builder = builder.config(key, value)
                
        spark = builder.getOrCreate()
        
        # Log cấu hình Spark
        self.logger.info("Spark Configuration:")
        for item in spark.sparkContext.getConf().getAll():
            self.logger.info(f"{item[0]}: {item[1]}")
            
        return spark

    def verify_input_file(self, input_path: str) -> bool:
        """
        Kiểm tra file input tồn tại và có thể đọc được
        """
        try:
            path = Path(input_path)
            if not path.exists():
                self.logger.error(f"File không tồn tại: {input_path}")
                return False
                
            # Thử đọc vài dòng đầu
            with open(input_path, 'r', encoding='utf-8') as f:
                for i, _ in enumerate(f):
                    if i >= 5: break  # Chỉ đọc 5 dòng đầu
                    
            return True
        except Exception as e:
            self.logger.error(f"Lỗi khi kiểm tra file input: {str(e)}")
            return False

    def verify_output_path(self, output_path: str) -> bool:
        """
        Kiểm tra và tạo thư mục output
        """
        try:
            path = Path(output_path)
            # Tạo thư mục nếu chưa tồn tại
            path.mkdir(parents=True, exist_ok=True)
            
            # Kiểm tra quyền ghi
            test_file = path / "test_write.tmp"
            test_file.touch()
            test_file.unlink()
            
            return True
        except Exception as e:
            self.logger.error(f"Lỗi khi kiểm tra thư mục output: {str(e)}")
            return False

    def cleanup(self) -> None:
        """Dọn dẹp tài nguyên Spark"""
        try:
            self.spark.stop()
            self.logger.info("Đã dừng SparkSession")
        except Exception as e:
            self.logger.error(f"Lỗi khi dừng SparkSession: {str(e)}")
            
    def define_schema(self) -> Optional[StructType]:
        """
        Định nghĩa schema cho Yelp Business dataset.
        Returns None nếu không cần validate schema.
        """
        return None
    def process_json_to_parquet(
        self,
        input_path: str,
        output_path: str,
        partition_columns: List[str] = None
    ) -> None:
        """
        Xử lý file JSON Yelp và lưu thành Parquet với kiểm tra chi tiết
        """
        try:
            # Kiểm tra input/output
            if not self.verify_input_file(input_path):
                raise ValueError(f"Invalid input file: {input_path}")
            if not self.verify_output_path(output_path):
                raise ValueError(f"Invalid output path: {output_path}")
                
            self.logger.info(f"Bắt đầu xử lý file: {input_path}")
            
            # Đọc một phần nhỏ dữ liệu để kiểm tra
            test_df = self.spark.read.json(
                input_path,
                schema=self.define_schema(),
                samplingRatio=0.1  # Chỉ đọc 10% dữ liệu để test
            )
            
            self.logger.info(f"Schema của DataFrame:")
            test_df.printSchema()
            
            # Đọc toàn bộ dữ liệu
            df = self.spark.read.json(
                input_path,
                schema=self.define_schema(),
                multiLine=False
            )
            
            # Log số lượng partitions
            partition_count = df.rdd.getNumPartitions()
            self.logger.info(f"Số lượng partitions: {partition_count}")
            
            # Thêm metadata
            df = df.withColumn("ingestion_timestamp", current_timestamp())\
                   .withColumn("source_file", lit(input_path))\
                   .withColumn("batch_id", lit(datetime.now().strftime("%Y%m%d_%H%M%S")))
            
            # Kiểm tra dữ liệu trước khi lưu
            row_count = df.count()
            self.logger.info(f"Số lượng records: {row_count}")
            
            if row_count == 0:
                raise ValueError("Không có dữ liệu để xử lý")
            
            # Lưu từng partition riêng biệt để dễ debug
            if partition_columns:
                # Kiểm tra partition columns tồn tại
                missing_cols = [col for col in partition_columns if col not in df.columns]
                if missing_cols:
                    raise ValueError(f"Các cột partition không tồn tại: {missing_cols}")
                
                self.logger.info(f"Partition theo các cột: {partition_columns}")
                
                writer = df.write\
                    .mode("append")\
                    .format("parquet")\
                    .option("compression", "snappy")\
                    .partitionBy(partition_columns)
            else:
                writer = df.write\
                    .mode("append")\
                    .format("parquet")\
                    .option("compression", "snappy")
            
            # Thử lưu một phần nhỏ dữ liệu trước
            test_output = f"{output_path}_test"
            test_df.write\
                .mode("overwrite")\
                .format("parquet")\
                .save(test_output)
                
            self.logger.info("Test write thành công, tiến hành lưu toàn bộ dữ liệu")
            
            # Lưu toàn bộ dữ liệu
            writer.save(output_path)
            
            # Kiểm tra kết quả
            result_df = self.spark.read.parquet(output_path)
            result_count = result_df.count()
            
            if result_count != row_count:
                raise ValueError(f"Số lượng records không khớp: {result_count} != {row_count}")
                
            self.logger.info(f"Đã xử lý và lưu thành công {result_count} records")
            
        except Exception as e:
            self.logger.error(f"Lỗi chi tiết khi xử lý dữ liệu: {str(e)}", exc_info=True)
            raise

def main():
    # Cấu hình Spark với ít memory hơn và ít partitions hơn
    spark_config = {
        "spark.driver.memory": "2g",
        "spark.executor.memory": "2g",
        "spark.sql.shuffle.partitions": "10",
        "spark.default.parallelism": "10"
    }
    
    processor = YelpDataProcessor(spark_config)
    
    try:
        input_path = "D:/Project/data/yelp_academic_dataset_business.json"
        output_path = "D:/Project/delta_lake/bronze/yelp_data/"
        
        # Thử không dùng partition trước
        processor.process_json_to_parquet(
            input_path=input_path,
            output_path=output_path
        )
        
    except Exception as e:
        processor.logger.error(f"Lỗi trong main: {str(e)}", exc_info=True)
    finally:
        processor.cleanup()

if __name__ == "__main__":
    main()

2025-01-16 12:12:59,415 - py4j.java_gateway - DEBUG - GatewayClient.address is deprecated and will be removed in version 1.0. Use GatewayParameters instead.
2025-01-16 12:12:59,417 - py4j.clientserver - DEBUG - Command to send: A
1320099b229fb756689f639ea33f2f0765c7d89e376e0d856a9ba7f31a0d0d35

2025-01-16 12:12:59,432 - py4j.clientserver - DEBUG - Answer received: !yv
2025-01-16 12:12:59,434 - py4j.clientserver - DEBUG - Command to send: j
i
rj
org.apache.spark.SparkConf
e

2025-01-16 12:12:59,437 - py4j.clientserver - DEBUG - Answer received: !yv
2025-01-16 12:12:59,438 - py4j.clientserver - DEBUG - Command to send: j
i
rj
org.apache.spark.api.java.*
e

2025-01-16 12:12:59,439 - py4j.clientserver - DEBUG - Answer received: !yv
2025-01-16 12:12:59,439 - py4j.clientserver - DEBUG - Command to send: j
i
rj
org.apache.spark.api.python.*
e

2025-01-16 12:12:59,440 - py4j.clientserver - DEBUG - Answer received: !yv
2025-01-16 12:12:59,441 - py4j.clientserver - DEBUG - Command to send: j
i
r

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

2025-01-16 12:13:07,563 - py4j.clientserver - DEBUG - Command to send: m
d
o116
e

2025-01-16 12:13:07,564 - py4j.clientserver - DEBUG - Answer received: !yv
2025-01-16 12:13:08,406 - py4j.clientserver - DEBUG - Answer received: !yro118
2025-01-16 12:13:08,407 - py4j.clientserver - DEBUG - Command to send: c
o118
javaToPython
e

2025-01-16 12:13:08,472 - py4j.clientserver - DEBUG - Answer received: !yro119
2025-01-16 12:13:08,473 - py4j.clientserver - DEBUG - Command to send: c
o119
id
e

2025-01-16 12:13:08,478 - py4j.clientserver - DEBUG - Answer received: !yi13
2025-01-16 12:13:08,479 - py4j.clientserver - DEBUG - Command to send: c
o119
partitions
e

2025-01-16 12:13:08,480 - py4j.clientserver - DEBUG - Answer received: !ylo120
2025-01-16 12:13:08,480 - py4j.clientserver - DEBUG - Command to send: c
o120
size
e

2025-01-16 12:13:08,481 - py4j.clientserver - DEBUG - Answer received: !yi10
2025-01-16 12:13:08,482 - __main__ - INFO - Số lượng partitions: 10
2025-01-16 12:13:08,482 - p

In [2]:
input_path = "D:/Project/data/yelp_academic_dataset_business.json"
output_path = "D:/Project/delta_lake/bronze/yelp_data/"